In [1]:
# ____ hitting all question in dataset ____ 
import pandas as pd
import numpy as np
import os

# get path to data C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\Synergistic_Computing_hf\benchmarking\new\dataset\MATH\test
path = r'C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\Synergistic_Computing_hf\benchmarking\new\dataset\MATH\test'

# get list of the sub directorys in directory
sub_dir = os.listdir(path)

# create a dataframe to store the contents of the files
dataset = pd.DataFrame(columns=['problem', 'solution'])

# read in the contents of 71 files from each sub directory and print the contents
for i in sub_dir:
    # get path to sub directory
    sub_dir_path = os.path.join(path, i)
    # get list of files in sub directory
    files = os.listdir(sub_dir_path)
    # get path to each file in sub directory. only read in the first 71 files
    for j in files[:172]:
        file_path = os.path.join(sub_dir_path, j)
        # the contents of the file is a json object and i only want the 'problem' and 'solution' keys
        df = pd.read_json(file_path, orient='index', typ='series', encoding='utf-8')
        # append the contents of the file to the dataframe
        dataset = dataset.append(df[['problem', 'solution']], ignore_index=True)

# how many questions are in the dataset
print(len(dataset))


C:\Users\cayde\AppData\Local\Temp\ipykernel_3164\3069383825.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(df[['problem', 'solution']], ignore_index=True)
C:\Users\cayde\AppData\Local\Temp\ipykernel_3164\3069383825.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(df[['problem', 'solution']], ignore_index=True)
C:\Users\cayde\AppData\Local\Temp\ipykernel_3164\3069383825.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(df[['problem', 'solution']], ignore_index=True)
C:\Users\cayde\AppData\Local\Temp\ipykernel_3164\3069383825.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Us

1204


C:\Users\cayde\AppData\Local\Temp\ipykernel_3164\3069383825.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(df[['problem', 'solution']], ignore_index=True)
C:\Users\cayde\AppData\Local\Temp\ipykernel_3164\3069383825.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(df[['problem', 'solution']], ignore_index=True)
C:\Users\cayde\AppData\Local\Temp\ipykernel_3164\3069383825.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(df[['problem', 'solution']], ignore_index=True)
C:\Users\cayde\AppData\Local\Temp\ipykernel_3164\3069383825.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Us

In [2]:
# keep only first 1200 questions
dataset = dataset[:1200]

In [3]:
# # ____ creating a prompt for the agent to answer ____

# # read in this json file C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\Synergistic_Computing_hf\benchmarking\new\dataset\MATH\test\algebra\1.json
# df = pd.read_json(r'C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\Synergistic_Computing_hf\benchmarking\new\dataset\MATH\test\algebra\1.json', orient='index', typ='series', encoding='utf-8')

# example_problem = df['problem']
# print(example_problem)
# # # create a prompt to send to agent. the prompt should explain that the agent is taking a test and that the agent should answer the question. and note the the equations are written in latex. the prompt should also contain the problem
# # prompt = 'You are taking a test. Answer the math questions to the best of your ability. Equations are written in latex. \n\n' + df['problem'] + '\n\n'

# # print(prompt)

In [4]:
# ____ get openai api key ____
# Path: C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\gpt-4-stuff\openai_key.txt
# open the file and read in the key
with open(r'C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\gpt-4-stuff\openai_key.txt', 'r') as f:
    openai_api_key = f.read()


In [5]:
# ____ init agent ____
# init langchain agent with gpt-4
from langchain import LLMChain, PromptTemplate, LLMMathChain
from langchain.llms import OpenAI
from langchain.agents import load_tools, initialize_agent, ZeroShotAgent, Tool, AgentExecutor
from langchain.chat_models import ChatOpenAI
from langchain.utilities import PythonREPL
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.chains import ConversationChain


llm = ChatOpenAI(model='gpt-4',temperature=0, openai_api_key=openai_api_key)
wolfram = WolframAlphaAPIWrapper(wolfram_alpha_appid='LXATGL-RJ6P2UWYT4')
python_repl = PythonREPL()

# ____ init tools ____
llm1 = OpenAI(temperature=0, openai_api_key=openai_api_key)
llm_math_chain = LLMMathChain(llm=llm1, verbose=True)
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer simple questions about math"
    ),
    Tool(
        name="python_repl", 
        func=python_repl.run,
        description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`."
    ),
    Tool(
        name="wolfram_alpha",
        func=wolfram.run,
        description="A tool that can answer questions about math, science, and more. Input should be a question. For example, `What is the derivative of x^2?`"
    )
]

# ____ init prompt ____
prefix = """You are taking a test. Answer the math questions to the best of your ability. Equations are written in latex. You have access to the following tools:"""
suffix = """Begin! Remeber answer the math questions to the best of your ability. Equations are written in latex.

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

# ____ run agent ____
llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

intermediate_steps = []
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True, max_iterations=7)

In [6]:
# # test agent is working
# response = agent_executor({'input':"what is 1+1?"})

# # how to grab the agents intermediate steps
# print(response["intermediate_steps"])

In [7]:
# ____ init gpt-4 ____
import openai

def ask_gpt4_student(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": """You are taking a test. Answer the math questions to the best of your ability. Equations are written in latex.""",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    assistant_reply = response["choices"][0]["message"]["content"]
    return assistant_reply

def ask_gpt4_grader(question, student_answer, actual_answer):
    prompt = "question: " + question + '\nstudent_answer: ' + student_answer + '\nactual_answer: ' + actual_answer


    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": """You are evaluating a student's work. you are provided the question the student was asked, and the student's work and answer, as well as the actual answer to the question. You are to evaluate the student's work and determine if the student was CORRECT or INCORRECT. Start each response with either CORRECT or INCORRECT explain briefly why the student is right or wrong. Equations are written in latex. """,
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    assistant_reply = response["choices"][0]["message"]["content"]
    return assistant_reply

In [8]:
# # test gpt-4 is working
# response = ask_gpt4("what is 1+1?")
# print(response)

In [9]:
# # test gpt-4 grader is working
# response = ask_gpt4_grader("what is 1+1?", "1+1=2", "2")
# print(response)

In [10]:
# im gonna feed batches of 120 questions to the agent. just to make sure i dont break rate limits. https://platform.openai.com/account/rate-limits
# break up the dataset into batches of 120 questions. these should be 10 dataframes of 120 questions each.

# grab 120 questions from the dataset and save them to a new dataframe and remove them from the original dataset
batch1 = dataset.sample(n=120)
# # add a column to the new dataframe that is called batch number
batch1['batch_number'] = 1
dataset = dataset.drop(batch1.index)

batch2 = dataset.sample(n=120)
batch2['batch_number'] = 2
dataset = dataset.drop(batch2.index)

batch3 = dataset.sample(n=120)
batch3['batch_number'] = 3
dataset = dataset.drop(batch3.index)

batch4 = dataset.sample(n=120)
batch4['batch_number'] = 4
dataset = dataset.drop(batch4.index)

batch5 = dataset.sample(n=120)
batch5['batch_number'] = 5
dataset = dataset.drop(batch5.index)

batch6 = dataset.sample(n=120)
batch6['batch_number'] = 6
dataset = dataset.drop(batch6.index)

batch7 = dataset.sample(n=120)
batch7['batch_number'] = 7
dataset = dataset.drop(batch7.index)

batch8 = dataset.sample(n=120)
batch8['batch_number'] = 8
dataset = dataset.drop(batch8.index)

batch9 = dataset.sample(n=120)
batch9['batch_number'] = 9
dataset = dataset.drop(batch9.index)

batch10 = dataset.sample(n=120)
batch10['batch_number'] = 10
dataset = dataset.drop(batch10.index)

In [29]:
# combine the batches into one dataframe called random_sample. add another column to the dataframe called batch_number. this will be used to keep track of which batch the question came from.
batch1['batch_number'] = 1
batch2['batch_number'] = 2
batch3['batch_number'] = 3
batch4['batch_number'] = 4
batch5['batch_number'] = 5
batch6['batch_number'] = 6
batch7['batch_number'] = 7
batch8['batch_number'] = 8
batch9['batch_number'] = 9
batch10['batch_number'] = 10
random_sample = pd.concat([batch1, batch2, batch3, batch4, batch5, batch6, batch7, batch8, batch9, batch10])

# write to csv
random_sample.to_csv('random_sample.csv')

C:\Users\cayde\AppData\Local\Temp\ipykernel_3164\2061464776.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch1['batch_number'] = 1


In [11]:
dataset = batch1
# save the index to a new column called 'PK'
dataset['PK'] = dataset.index
# reset the index of the dataset
dataset = dataset.reset_index(drop=True)

In [12]:
# add to 'dataset' dataframe add empty columns for agents intermediate steps and the agents answer. also add a column called gpt-4 answer.
dataset['agent_intermediate_steps'] = ''
dataset['agent_answer'] = ''
dataset['agents_score'] = ''
dataset['gpt-4_answer'] = ''
dataset['gpt-4_score'] = ''

In [26]:
# remove the first 24 rows from the dataset.
dataset = dataset.iloc[56:]
# reset the index of the dataset
dataset = dataset.reset_index(drop=True)

In [23]:
# ____ run agent on dataset ____
for i in range(0, len(dataset)):
    # get problem from dataset
    problem = dataset['problem'][i]
    # get solution from dataset
    solution = dataset['solution'][i]
    try:
        # get the agents response
        response = agent_executor({'input':problem})
        # get the agents intermediate steps
        intermediate_steps = response["intermediate_steps"]
        # get the agents answer
        agent_answer = response["output"]
        # format agent answer and work for grading
        agent_intermediate_steps = str(dataset['agent_intermediate_steps'][i])
        agent_answer = str(dataset['agent_answer'][i])
        agent_scratchpad = agent_intermediate_steps + agent_answer
        # grade the agents answer
        agents_score = ask_gpt4_grader(problem, agent_scratchpad, solution)
    except Exception as e:
        agent_answer = 'failed to answer'
        agents_score = 'INCORRECT'
        print(f"Error with agent: {e}")

    # get the gpt-4 answer
    gpt_4_answer = ask_gpt4_student(problem)
    # grade the gpt-4 answer
    gpt_4_score = ask_gpt4_grader(problem, gpt_4_answer, solution)
    # add the agents intermediate steps to the dataset
    dataset['agent_intermediate_steps'][i] = intermediate_steps
    # add the agents answer to the dataset
    dataset['agent_answer'][i] = agent_answer
    # add the gpt-4 answer to the dataset
    dataset['gpt-4_answer'][i] = gpt_4_answer
    # add the agents score to the dataset
    dataset['agents_score'][i] = agents_score
    # add the gpt-4 score to the dataset
    dataset['gpt-4_score'][i] = gpt_4_score



> Entering new AgentExecutor chain...
Thought: I need to find the radius of the red circles. I will use Descartes' Circle Theorem to find the radius.
Thought: Descartes' Circle Theorem states that if four circles are mutually tangent, then the sum of the curvatures squared is twice the sum of the product of the pairs of curvatures. The curvature of a circle is the reciprocal of its radius.
Action: Calculator
Action Input: 1/1 + 1/1 + 1/1

> Entering new LLMMathChain chain...
1/1 + 1/1 + 1/1Answer: 3
> Finished chain.

Observation: Answer: 3
Thought:The sum of the curvatures of the three unit circles is 3. Now I need to find the curvature of the blue circle.
Thought: The blue circle is tangent to all three unit circles, so its curvature is 3 minus the curvature of each unit circle.
Action: Calculator
Action Input: 3 - 1

> Entering new LLMMathChain chain...
3 - 1Answer: 2
> Finished chain.

Observation: Answer: 2
Thought:The curvature of the blue circle is 2. Now I need to find the cu

C:\Users\cayde\AppData\Local\Temp\ipykernel_3164\186015321.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['agent_intermediate_steps'][i] = intermediate_steps
C:\Users\cayde\AppData\Local\Temp\ipykernel_3164\186015321.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['agent_answer'][i] = agent_answer
C:\Users\cayde\AppData\Local\Temp\ipykernel_3164\186015321.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d



> Entering new AgentExecutor chain...
Thought: I need to find the number of integers between 200 and 300 that have a digit sum of 15.
Thought: I can use a Python script to iterate through the numbers and check the digit sum.
Action: python_repl
Action Input: 
count = 0
for i in range(200, 301):
    digits = [int(d) for d in str(i)]
    if sum(digits) == 15:
        count += 1
print(count)
Observation: 6

Thought:I now know the final answer.
Final Answer: 6

> Finished chain.


> Entering new AgentExecutor chain...
Thought: I need to find the total number of handshakes between 10 people. This is a combination problem.
Action: wolfram_alpha
Action Input: How many combinations of 10 people taken 2 at a time?
Observation: Wolfram Alpha wasn't able to answer it
Thought:I can use the combination formula to calculate the number of handshakes: C(n, k) = n! / (k!(n-k)!) where n = 10 and k = 2.
Action: python_repl
Action Input: from math import comb; print(comb(10, 2))
Observation: 45

Thought

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Tue, 30 May 2023 20:54:13 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7cf9d70f083a32e9-JAX', 'alt-svc': 'h3=":443"; ma=86400'}.


I made a mistake in my calculation again. I need to recalculate the probability that Hosea will draw a white marble.
Action: Calculator
Action Input: 6 / (28 - 1)

> Entering new LLMMathChain chain...
6 / (28 - 1)Answer: 2
> Finished chain.

Observation: Answer: 2
Thought:I made a mistake in my calculation again. I need to recalculate the probability that Hosea will draw a white marble.
Action: Calculator
Action Input: 6 / (28 - 1)

> Entering new LLMMathChain chain...
6 / (28 - 1)Answer: 2
> Finished chain.

Observation: Answer: 2
Thought:

> Finished chain.


> Entering new AgentExecutor chain...
Thought: I know that a pyramid has one more vertex than the number of edges in its base. I can use the formula for the number of edges in a polygon to find the number of vertices in the base, and then add one to find the total number of vertices in the pyramid.
Action: wolfram_alpha
Action Input: How many vertices are in a polygon with 14 edges?
Observation: Assumption: tetradecagon | vertic

APIError: HTTP code 502 from API (<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>
)

In [24]:
# # grab the first 24 rows of the dataset and save them to a new dataframe
batch1 = dataset.head(56)


In [25]:
# ____ save dataset ____
# save dataset to cwd
batch1.to_csv('output/batch1_Q_33_89.csv', index=False)

In [28]:
# ___ add a rety api call to script __

import time
import pandas as pd
import os

MAX_RETRIES = 5

def retry_on_failure(function, *args, **kwargs):
    for attempt in range(MAX_RETRIES):
        try:
            return function(*args, **kwargs)
        except Exception as e:
            if attempt < MAX_RETRIES - 1:  # i.e., not on last attempt
                sleep_seconds = 2 ** attempt  # exponential backoff
                print(f"Error: {e}. Retrying in {sleep_seconds} seconds...")
                time.sleep(sleep_seconds)
            else:
                print(f"Error: {e}. No more retries.")
                raise e

for i in range(len(dataset)):
    problem = dataset.loc[i, 'problem']
    solution = dataset.loc[i, 'solution']

    try:
        response = retry_on_failure(agent_executor, {'input': problem})
        intermediate_steps = str(response["intermediate_steps"])
        agent_answer = str(response["output"])
        # agent_intermediate_steps = str(dataset.loc[i, 'agent_intermediate_steps'])
        # agent_answer = str(dataset.loc[i, 'agent_answer'])
        agent_scratchpad = str(agent_intermediate_steps + agent_answer)
        agents_score = retry_on_failure(ask_gpt4_grader, problem, agent_scratchpad, solution)
    except Exception as e:
        agent_answer = 'failed to answer'
        agents_score = 'INCORRECT'
        print(f"Error with agent: {e}")

    try:
        gpt_4_answer = retry_on_failure(ask_gpt4_student, problem)
        gpt_4_score = retry_on_failure(ask_gpt4_grader, problem, gpt_4_answer, solution)
    except Exception as e:
        gpt_4_answer = 'failed to answer'
        gpt_4_score = 'INCORRECT'
        print(f"Error with GPT-4: {e}")

    dataset.loc[i, 'agent_intermediate_steps'] = intermediate_steps
    dataset.loc[i, 'agent_answer'] = agent_answer
    dataset.loc[i, 'agents_score'] = agents_score
    dataset.loc[i, 'gpt-4_answer'] = gpt_4_answer
    dataset.loc[i, 'gpt-4_score'] = gpt_4_score

    dataset.to_csv('output/batch1_Q_89_120.csv', index=False)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)).




> Entering new AgentExecutor chain...
Thought: I need to find the projection of vector v onto -w. I know that the projection of v onto w is given by the formula: proj_w(v) = (v . w) / ||w||^2 * w. Since I already have the projection of v onto w, I can use the fact that proj_{-w}(v) = -proj_w(v) to find the projection of v onto -w.
Action: Calculator
Action Input: -1 * (2, -11)

> Entering new LLMMathChain chain...
-1 * (2, -11)Answer: (-2, 11)
> Finished chain.

Observation: Answer: (-2, 11)
Thought:I now know the final answer.
Final Answer: (-2, 11)

> Finished chain.


> Entering new AgentExecutor chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: HTTP code 502 from API (<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>
).


Thought: I need to simplify the given expression. I will use the Python shell to compute the values.
Action: python_repl
Action Input: from sympy import *
i = I
expr1 = ((3 + i*sqrt(3))/2)**8
expr2 = ((3 - i*sqrt(3))/2)**8
result = expr1 + expr2
simplified_result = simplify(result)
print(simplified_result)
Observation: No module named 'sympy'
Thought:I cannot use sympy in this environment. I will try using Wolfram Alpha instead.
Action: wolfram_alpha
Action Input: Simplify ((3 + i*sqrt(3))/2)^8 + ((3 - i*sqrt(3))/2)^8
Observation: Assumption: simplify | (1/2 (3 + i sqrt(3)))^8 + (1/2 (3 - i sqrt(3)))^8 
Answer: -81/2 (sqrt(3) - 1) (1 + sqrt(3))
Thought:I now know the final answer.
Final Answer: -81/2 (sqrt(3) - 1) (1 + sqrt(3))

> Finished chain.
Error: HTTP code 502 from API (<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>
). Retrying in 1 seconds...


> Entering new AgentExecutor chain

In [ ]:
# save random sample that is up a couple cells so u dont loose batches 